In [1]:
:opt no-lint
{-# LANGUAGE Arrows #-}
--{-# LANGUAGE DataKinds #-}
{-# LANGUAGE GADTs #-}
{-# LANGUAGE OverloadedStrings   #-}
{-# LANGUAGE QuasiQuotes         #-}
--{-# LANGUAGE RankNTypes #-}
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE TypeOperators       #-}

{-# LANGUAGE PolyKinds             #-}
{-# LANGUAGE Rank2Types            #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE FlexibleInstances #-}
{-# LANGUAGE ExistentialQuantification #-}
{-# LANGUAGE ViewPatterns              #-}

import Control.External (contentParam)
import Data.CAS.ContentHashable
import Data.Typeable
import qualified Data.CAS.ContentStore as CS
import Data.CAS.ContentStore (Content)
import qualified Data.CAS.RemoteCache as RC
import Funflow.Config
import Funflow (Flow, dockerFlow, ioFlow, pureFlow, RunFlowConfig(..), runFlow, runFlowWithConfig, RequiredCore, RequiredStrands)
import qualified Funflow.Tasks.Docker as DE

import Control.Arrow (returnA)
import Control.Monad (guard)
import Control.Monad.IO.Class (MonadIO)
import Control.Kernmantle.Rope (AnyRopeWith)

import Data.List (replicate)
import qualified Data.Map as Map
import qualified Data.Set as Set
import qualified Data.Text as Text
import Data.Traversable (mapM)
import Path (Abs, Dir, File, Path, Rel, filename, fromAbsFile, parseAbsDir, parseAbsFile, parseRelFile, relfile, toFilePath, (</>))

import System.IO (readFile)
import System.Posix.Files (accessModes, createLink, setFileMode)

type Set = Set.Set

In [2]:
type SourceFile = String     -- | Name of source file
type TargetFile = String     -- | Name of target file
type BuildFile  = String     -- | Either source xor target
type FileName = String
type FileContent = String

data MakeFile where
  MakeFile :: { sourceFiles :: Set SourceFile
              , defaultGoal :: MakeRule
              , allrules :: Set MakeRule
              } -> MakeFile
  deriving Show

data MakeRule where
  MakeRule :: TargetFile -> Set BuildFile -> Command -> MakeRule
  deriving (Eq, Ord, Show)

type Command = String

-- Makefile error type
newtype MFError = MFError String

mkRuleTarNm :: MakeRule -> String
mkRuleTarNm (MakeRule tf _ _) = tf

In [3]:
ioFixSrcFileData :: (FileName, FileContent) -> IO (FileContent, Path Rel File)
ioFixSrcFileData (x,y) = (\y' -> (y,y')) <$> parseRelFile x

In [4]:
putInStoreAtRaw :: (ContentHashable IO a, Typeable t) => Path Abs Dir -> (Path Abs t -> a -> IO ()) -> (a, Path Rel t) -> IO (CS.Content t)
putInStoreAtRaw root put (a,p) = CS.withStore root (\store -> do
    item <- CS.putInStore store RC.NoCache (\_ -> return ()) (\d a -> put (d </> p) a) a
    return (item CS.:</> p)
    )

In [5]:
write1 :: Path Abs Dir -> (FileName, FileContent) -> IO (Content File)
write1 d x@(fn, fc) = ioFixSrcFileData x >>= putInStoreAtRaw d (writeFile . fromAbsFile)

write2StoreRaw d = mapM (write1 d) . Map.toList

In [6]:
mergeFilesRaw :: Path Abs Dir -> [CS.Content File] -> IO (CS.Content Dir)
mergeFilesRaw root fs = CS.withStore root (\s -> merge s fs)
    where merge store files = let absFiles = map (CS.contentPath store) files
                                  linkIn d = mapM_ ( \f -> createLink (toFilePath f) (toFilePath $ d </> filename f) )
                              in CS.All <$> CS.putInStore store RC.NoCache (\_ -> return (error "uh-oh!")) linkIn absFiles

In [7]:
writeExecStrRaw :: Path Abs Dir -> (String, Path Rel File) -> IO (CS.Content File)
writeExecStrRaw d = putInStoreAtRaw d (\p x -> do
  writeFile (fromAbsFile p) x
  setFileMode (fromAbsFile p) accessModes)

In [8]:
-- | Compiles a C file in a docker container.
--compileFile :: Path Abs Dir -> Flow (TargetFile, Map.Map SourceFile String, [Content File], Command) (Content File)
--compileFile :: Path Abs Dir -> MyExpFlow (TargetFile, Map.Map SourceFile String, [Content File], Command) (Content File)
compileFile :: Path Abs Dir -> Flow (TargetFile, Map.Map SourceFile String, [Content File], Command) (Content File)
compileFile root = ioFlow ( \(tf, srcDeps, tarDeps, cmd) -> do
    srcsInStore <- write2StoreRaw root srcDeps
    let inputFilesInStore = srcsInStore ++ tarDeps
    inputDir <- mergeFilesRaw root inputFilesInStore
    let scriptSrc = "#!/usr/bin/env bash\n\
                    \cd $1 \n"
                    ++ cmd ++ " -o /output/" ++ tf
    compileScript <- writeExecStrRaw root (scriptSrc, [relfile|script.sh|])
    relpathCompiledFile <- parseRelFile tf
    let runCfg = RunFlowConfig{ configFile = Nothing, storePath = root }
    let depIt = CS.contentItem inputDir :: CS.Item
    let gccCfg compile = DE.DockerTaskConfig{
      DE.image = "gcc:7.3.0", 
      DE.command = Text.pack . toFilePath $ (root </> CS.contentFilename compile), 
      DE.args = [DE.Arg . Literal . Text.pack . toFilePath $ (CS.itemRelPath depIt)] } --[contentParam depDir]}    
    (CS.:</> relpathCompiledFile) <$> runFlowWithConfig runCfg (dockerFlow (gccCfg compileScript)) (mempty :: DE.DockerTaskInput) )
    --compiledFile <- runFlowWithConfig cfg gcc (mempty :: DE.DockerTaskInput) :: IO CS.Item
    --return (compiledFile CS.:</> relpathCompiledFile) )


-- Note: type SourceFile = String.
-- Note: TargetFile is the name of the file.


In [9]:
--buildTarget :: Path Abs Dir -> MakeFile -> MakeRule -> Flow () (Content File)
--buildTarget :: MonadIO m => Path Abs Dir -> MakeFile -> MakeRule -> AnyRopeWith RequiredStrands (RequiredCore m) () (Content File)
buildTarget :: Path Abs Dir -> MakeFile -> MakeRule -> IO (Content File)
buildTarget storeRoot mkfile target@(MakeRule targetNm deps cmd) = let
   srcfiles = sourceFiles mkfile
   neededTargets = Set.toList $ Set.difference deps srcfiles
   neededSources  = Set.toList $ deps `Set.intersection` srcfiles
   maybeFindDepRules = findRules mkfile neededTargets  
 in case maybeFindDepRules of
   Nothing -> return (error "dependency rules missing")
   Just (depRules :: [MakeRule]) -> let
       depTargetActions = mapM (buildTarget storeRoot mkfile) depRules
       grabSrcsActions = mapM (readFile . ("./" ++))
     in do
       guard (target `Set.member` (allrules mkfile))
       contentSrcFiles <- grabSrcsActions neededSources
       depFiles <- depTargetActions
       --depFiles <- flowJoin [Id { unId = f } | f <- depTargetFlows] -< ( replicate (length depTargetFlows) () )
       --let depsFlow = (flowJoin depTargetFlows) :: Flow [()] [Content File]
       --depFiles <- depsFlow -< ( replicate (length depTargetFlows) () ) :: IO (Content File)
       --depFiles <- ioFlow ( \_ -> mapM (\Id{ unId = f} -> runFlow f () :: IO (Content File)) depTargetFlows ) -< ()
       let fullSrcFiles = Map.fromList $ zip neededSources contentSrcFiles
       compFile <- runFlow (compileFile storeRoot) (targetNm, fullSrcFiles, depFiles,cmd)
       return compFile

findRules :: MakeFile -> [TargetFile] -> Maybe [MakeRule]
findRules MakeFile{ allrules = rules } ts = do
  guard . null $ Set.difference tfileSet ruleTarNmSet
  let targetRules = Set.filter ((`Set.member` tfileSet) . mkRuleTarNm) rules
  return $ Set.toList targetRules
  where
    ruleTarNmSet = Set.map mkRuleTarNm rules
    tfileSet = Set.fromList ts


In [10]:
--type MyExpFlow input output =
--  AnyRopeWith
--    RequiredStrands
--    (RequiredCore IO)
--    input
--    output

--newtype Id a b = Id{ unId :: MyExpFlow a b }
--type FlowList a b = [Id a b]

--runFlowListHom :: RunFlowConfig -> FlowList a b -> a -> IO [b]
--runFlowListHom runCfg flows homIn = sequence . reverse $ go flows []
--    where go [] acc = acc
--          go (f:fs) acc = (runFlowWithConfig runCfg (unId f) homIn) : acc

In [11]:
guardFlow :: Flow Bool ()
guardFlow = ioFlow (\p -> if p then return () else return (error "error"))

-- https://stackoverflow.com/questions/56448814/why-is-impredicative-polymorphism-allowed-only-for-functions-in-haskell
newtype Id a b = Id{ unId :: (Flow a b) }
type FlowList a b = [Id a b]

type MyExpFlow input output =
  forall m.
  (MonadIO m) =>
  AnyRopeWith
    RequiredStrands
    (RequiredCore m)
    input
    output

flowJoin :: FlowList a b -> Flow [a] [b]
--flowJoin = undefined
flowJoin [] = pureFlow (\_ -> [])
flowJoin ff@(f:fs) = proc aa@(a:as) -> do
  () <- guardFlow -< (length ff == length aa)
  b <- unId f -< a
  bs <- flowJoin fs -< as
  returnA -< (b:bs)

----flowJoin :: [a ==> b] -> ([a] ==> [b])
--flowJoin :: [Flow () (Content File)] -> Flow () [Content File]
--flowJoin [] = step (\_ -> [])
--flowJoin ff@(f:fs) = proc aa@(a:as) -> do
--  () <- guardFlow -< (length ff == length aa)
--  b <- f -< a
--  bs <- flowJoin fs -< as
--  returnA -< (b:bs)


--flows2Files :: [Flow () (Content File)] -> (Flow () [Content File])
--flows2Files flows = go flows []
--    where go [] acc = pureFlow (\_ -> reverse acc)
--          go (f:fs) acc = go fs (let res = runFlow f () in res:acc)



-- error:
--    • Illegal polymorphic type:
--        forall (m :: * -> *). Control.Monad.IO.Class.MonadIO m => Control.Kernmantle.Rope.AnyRopeWith ('[] Funflow.Type.Family.List.++ Funflow.Flow.RequiredStrands) (Funflow.Flow.RequiredCore m) a b
--      GHC doesn't yet support impredicative polymorphism
--    • In the expansion of type synonym ‘Funflow.Flow.ExtendedFlow’
--      In the expansion of type synonym ‘Flow’
--      In the type signature: flowJoin :: [Flow a b] -> Flow [a] [b]

--yieldFiles :: [Flow () (Content File)] -> [Content File]
--yieldFiles flows = reverse $ go flows []
--    where go [] acc = acc
--          go (f:fs) = do